In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [ ]:
path = '/kaggle/input/bookcrossing-dataset/Book reviews/'
users_data = pd.read_csv(path+"BX-Users.csv", sep=";", error_bad_lines=False,encoding='latin-1') # 'ISO-8859-1'
books_data = pd.read_csv(path+"BX_Books.csv", sep=";", error_bad_lines=False, encoding='latin-1')
rating_data = pd.read_csv(path+"BX-Book-Ratings.csv", sep=";", error_bad_lines=False, encoding='latin-1')

In [ ]:
books_data.head()

In [ ]:
rating_data.head()

Delete unusable information from books frame and merge with ratings frame

In [ ]:
books_data = books_data.drop(['Year-Of-Publication', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1)
books_rating_data = books_data.merge(rating_data, on='ISBN')
books_rating_data

Add the number of assigned ratings. Clear from users who have rated a small number of times

In [ ]:
number_of_ratings = books_rating_data.groupby('ISBN')['Book-Rating'].count().reset_index()
number_of_ratings.rename(columns={'Book-Rating': 'Number of Book-Rating'}, inplace=True)
books_rating_data = books_rating_data.merge(number_of_ratings, on='ISBN')
books_rating_data

In [ ]:
books_rating_data = books_rating_data[books_rating_data['Number of Book-Rating'] >= 30]
books_rating_data.shape

In [ ]:
books_rating_data.drop_duplicates(['User-ID', 'Book-Title'], inplace=True)
books_rating_data.shape

Making сolloaborative кecommendation system

In [ ]:
books_rating_pivot = books_rating_data.pivot_table(columns='User-ID', index='Book-Title', values='Book-Rating')
books_rating_pivot.fillna(0, inplace=True)
books_rating_pivot

In [ ]:
book_csr_matrix = csr_matrix(books_rating_pivot)

In [ ]:
model = NearestNeighbors(algorithm='brute')
model.fit(book_csr_matrix)

In [ ]:
books_rating_pivot.iloc[54, :].values.reshape(1,-1)

In [ ]:
_, recommendations = model.kneighbors(books_rating_pivot.iloc[54, :].values.reshape(1,-1))
recommendations

In [ ]:
for i in range(len(recommendations)):
    print(recommendations[i])
    print(books_rating_pivot.index[recommendations[i]])

In [ ]:
def recomend_book(book_name):
    book_id = np.where(books_rating_pivot.index == book_name)[0][0]
    _, recommendations = model.kneighbors(books_rating_pivot.iloc[book_id,:].values.reshape(1,-1))
    for i in range(len(recommendations)):
        if i == 0:
            print(f"For book \"{book_name}\" is recommended")
        if not i:
            print(books_rating_pivot.index[recommendations[i]])

Test recomendation

In [ ]:
recomend_book('Exclusive')